# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
#langchain library
!pip3 install langchain==0.1.0
#LLM
!pip3 install accelerate==0.25.0
!pip3 install transformers==4.36.2
!pip3 install bitsandbytes==0.41.2
#Text Embedding
!pip3 install sentence-transformers==2.2.2
!pip3 install InstructorEmbedding==1.0.1
#vectorstore
!pip3 install pymupdf==1.23.8
!pip3 install faiss-gpu==1.7.2
!pip3 install faiss-cpu==1.7.4

ERROR: Could not find a version that satisfies the requirement faiss-gpu==1.7.2 (from versions: none)
ERROR: No matching distribution found for faiss-gpu==1.7.2


In [2]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [3]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:")

In [4]:
PROMPT.format(
    context = "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.",
    question = "What is Machine Learning"
)

"I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [5]:
!pip3 install unstructured

In [6]:
# !pip3 install langchain==0.1.0

In [7]:
# from langchain. document_loaders import PyMuPDFLoader

# nlp_docs = 'datasets/2_TextProc_2023.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()

In [8]:
# multiple pdf

import os
from langchain.document_loaders import PyMuPDFLoader

# Path to the folder containing PDF documents
folder_path = 'datasets'

# Initialize an empty list to store loaded documents
all_documents = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a PDF
    if file_name.endswith('.pdf'):
        # Construct the full path to the PDF file
        pdf_path = os.path.join(folder_path, file_name)
        
        # Load the PDF document
        loader = PyMuPDFLoader(pdf_path)
        documents = loader.load()
        
        # Add the loaded documents to the list
        all_documents.extend(documents)

# Now, all_documents contains the loaded documents from all PDFs in the "datasets" folder


In [9]:
# documents
# documents = all_documents

In [10]:
len(all_documents)

405

In [11]:
documents[3]

Document(page_content='      New Trends for Future Services \nNext Generation Networks and Services \nWireless Access and Transport Technologies\nWireless Local Area and Personal Networks\nService-oriented Computing\nOptimisation: Theory and Network Applications\nBusiness Modelling for Mobile Services \nM.Sc. Thesis / Project\nYEAR ONE\nAIT, THAILAND\nTelecommunication\nComputer Science\nInformation Management\nInformation and Communication \nTechnology\nABOUT THE PROGRAM\nCommunication Network and Services (ComNETS)\nAIT and Telecom SudParis jointly offer a unique Euro-Asian curriculum at the master’s level \naddressing key issues in communication networks and services in the European and Asian contexts.  \nUpon successfully completing their studies in Asia and Europe, these students earn both an AIT \nmaster’s degree and an MSc degree from Telecom SudParis (TSP).  Graduates are ready to apply \ncutting-edge concepts and technologies in IT and the telecommunications industry.  Graduat

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [13]:
doc[1]

Document(page_content='In their first year of study, students have the opportunity of entering \none of four fields of study at AIT: Telecommunications, Information \nand Communication Technology, Computer Science, or Information \nManagement. In their second year at Telecom SudParis, they focus on \nrecent evolution in Wireless Networks, Future Internet, Next Generation \nNetworks and Services.', metadata={'source': 'datasets/AIT_TSP.pdf', 'file_path': 'datasets/AIT_TSP.pdf', 'page': 0, 'total_pages': 4, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5 (7.0)', 'producer': 'Adobe PDF Library 9.9', 'creationDate': "D:20130410145137+07'00'", 'modDate': "D:20130410145140+07'00'", 'trapped': ''})

In [14]:
len(doc)

13

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [15]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

ImportError: cannot import name '_is_openai_v1' from 'langchain_community.embeddings.openai' (/Users/myothiha/opt/anaconda3/envs/NLP_Datacamp/lib/python3.9/site-packages/langchain_community/embeddings/openai.py)

### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [ ]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [ ]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

KeyboardInterrupt: 

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [ ]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [ ]:
#ready to use
retriever = vectordb.as_retriever()

In [ ]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(page_content='for projective dependency parsing.\nProceedings of the 8th International\nWorkshop on Parsing Technologies\n(IWPT).\nNivre, J. 2006. Inductive Dependency\nParsing. Springer.\nNivre, J. 2009.\nNon-projective de-\npendency parsing in expected linear\ntime. ACL IJCNLP.', metadata={'source': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 614, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': "D:20230107092057-08'00'", 'modDate': "D:20230107092057-08'00'", 'trapped': ''}),
 Document(page_content='dependency parse; the internal structure of the dependency parse consists solely of\ndirected relations between words. These head-dependent relationships directly en-\ncode important information that is often buried in the more complex phrase-structure\n

In [ ]:
retriever.get_relevant_documents("What is Transformers")

[Document(page_content='formers are not based on recurrent connections (which can be hard to parallelize),\nwhich means that transformers can be more efﬁcient to implement at scale.\nTransformers map sequences of input vectors (x1,...,xn) to sequences of output\nvectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-\nformer blocks, each of which is a multilayer network made by combining simple\nlinear layers, feedforward networks, and self-attention layers, the key innovation of\nself-attention\ntransformers. Self-attention allows a network to directly extract and use information\nfrom arbitrarily large contexts without the need to pass it through intermediate re-', metadata={'source': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 219, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyper

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [ ]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [ ]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [ ]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

2024-01-02 14:57:27.374664: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 14:57:27.392320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 14:57:27.392344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 14:57:27.392979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 14:57:27.396491: I tensorflow/core/platform/cpu_feature_guar

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [ ]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [ ]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [ ]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing Both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing Both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [ ]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7fefa8f7ec50>)), document_variable_name='context')

In [ ]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    formers are not based on recurrent connections (which can be hard to parallelize),
which means that transformers can be more efﬁcient to implement at scale.
Transformers map sequences of input vectors (x1,...,xn) to sequences of output
vectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-
former blocks, each of which is

{'input_documents': [Document(page_content='formers are not based on recurrent connections (which can be hard to parallelize),\nwhich means that transformers can be more efﬁcient to implement at scale.\nTransformers map sequences of input vectors (x1,...,xn) to sequences of output\nvectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-\nformer blocks, each of which is a multilayer network made by combining simple\nlinear layers, feedforward networks, and self-attention layers, the key innovation of\nself-attention\ntransformers. Self-attention allows a network to directly extract and use information\nfrom arbitrarily large contexts without the need to pass it through intermediate re-', metadata={'source': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 219, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2Te

## 5. Chatbot

In [ ]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    in this way are also called complementizers.
complementizer
Pronouns act as a shorthand for referring to an entity or event. Personal pro-
pronoun
nouns refer to persons or entities (you, she, I, it, me, etc.). Possessive pronouns are
forms of personal pronouns that indicate either actual possess

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ',
 'source_documents': [Document(page_content='in this way are also called complementizers.\ncomplementizer\nPronouns act as a shorthand for referring to an entity or event. Personal pro-\npronoun\nnouns refer to persons or entities (you, she, I, it, me, etc.). Possessive pronouns are\nforms of personal pro

In [ ]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ')]
Follow Up Input: What is the Tran

{'question': 'What is the Transformers?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ')],
 'answer': '<pad>  transformers  are  a  type  of  neural  network  that  are  used  to  map  sequences  of  input  vectors  (x1,...,xn)  to  sequences  of  output  vectors  (y1,...,yn)  of  the  same  length.  They  are  made  up  of  stacks  of 

In [ ]:
prompt_question = "Is it a statistical model?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train '), HumanMessage(content='What is the

{'question': 'Is it a statistical model?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train '),
  HumanMessage(content='What is the Transformers?'),
  AIMessage(content='<pad>  transformers  are  a  type  of  neural  network  that  are  used  to  map  sequences  of  input  vectors  (x1,...,xn)  to  sequences  of  output  vectors  (y1,...,yn)